* > # EE226 - Coding 2
## Streaming algorithm & Locality Sensitive Hashing

### Streaming: DGIM

DGIM is an efficient algorithm in processing large streams. When it's infeasible to store the flowing binary stream, DGIM can estimate the number of 1-bits in the window. In this coding, you're given the *stream_data.txt* (binary stream), and you need to implement the DGIM algorithm to count the number of 1-bits. Write code and ask the problems below.

### Your task

1. Set the window size to 1000, and count the number of 1-bits in the current window.

In [ ]:
# Your code here, you can add cells if necessary
import time

In [ ]:
bucket_n=[]#桶的列表

window = 1000
time_location = 5000
#time_location = eval(input("请输入时刻："))
max_bucket = 2

def Is_due(time_now,bucket_n):
    if len(bucket_n) > 0 and time_now - window == bucket_n[0]['timestamp']:
        del bucket_n[0]
    
def Merge(bucket_n):
    for i in range(len(bucket_n)-1,max_bucket-1,-1):
        if bucket_n[i]['bit_sum'] == bucket_n[i - max_bucket]['bit_sum']:
            bucket_n[i - max_bucket]['bit_sum'] += bucket_n[i - max_bucket + 1]['bit_sum']
            bucket_n[i - max_bucket]['timestamp'] = bucket_n[i - max_bucket + 1]['timestamp']
            del bucket_n[i-max_bucket+1]

def Count_DGIM(bucket_n):
    bit_sum = 0

    with open("../input/coding2/stream_data.txt","rt") as f:
        for i in range(time_location):
            tmp = f.read(2)[0]
            if tmp:
                Is_due(i+1,bucket_n)#判断是否有桶到期
                if tmp == '1':
                    bucket={"timestamp":i+1,"bit_sum":1}#桶的结构
                    bucket_n.append(bucket)
                    Merge(bucket_n)#合并大小相同的桶
    for i in range(len(bucket_n)):
        bit_sum += bucket_n[i]['bit_sum']
    bit_sum -= bucket_n[0]['bit_sum']/2
    return bit_sum if len(bucket_n)>0 else 0

#开始时间 
start_time = time.time()
#程序运行
bit_sum=Count_DGIM(bucket_n)
#结束时间
end_time = time.time()
#计算运行时间
run_time = end_time - start_time 

print(bit_sum,run_time)

2. Write a function that accurately counts the number of 1-bits in the current window, and compare the difference between its running time and space and the DGIM algorithm.

In [ ]:
# Your code here, you can add cells if necessary
def Count():
    bit_sum = 0
    if time_location - window < 0:
        start_location = 0
    else:
        start_location = time_location - window
    with open("../input/coding2/stream_data.txt","rt") as f:
        for i in range(time_location):
            tmp = f.read(2)[0]
            if i >= start_location and tmp == '1':
                bit_sum += 1
    return bit_sum

    

#开始时间 
start_time = time.time()
#程序运行
bit_sum=Count()
#结束时间
end_time = time.time()
#计算运行时间
run_time = end_time - start_time 

print(bit_sum,run_time)

### Locality Sensitive Hashing

The locality sensitive hashing (LSH) algorithm is efficient in near-duplicate document detection. In this coding, you're given the *docs_for_lsh.csv*, where the documents are processed into set of k-shingles (k = 8, 9, 10). *docs_for_lsh.csv* contains 201 columns, where column 'doc_id' represents the unique id of each document, and from column '0' to column '199', each column represents a unique shingle. If a document contains a shingle ordered with **i**, then the corresponding row will have value 1 in column **'i'**, otherwise it's 0. You need to implement the LSH algorithm and ask the problems below.

### Your task

Use minhash algoirthm to create signature of each document, and find 'the most similar' documents under Jaccard similarity. 
Parameters you need to determine:
1) Length of signature (number of distinct minhash functions) *n*. Recommanded value: n > 20.

2) Number of bands that divide the signature matrix *b*. Recommanded value: b > n // 10.

### read the data

In [ ]:
# Your code here, you can add cells if necessary
import numpy as np
import pandas as pd
filepath = '../input/coding2/docs_for_lsh.csv'
data = pd.read_csv(filepath)
data = np.array(data)
data = data[:,1:]
print(data.shape)
print(data)

### get the Permutation π

In [ ]:
# Length of signature (number of distinct minhash functions) n
N = 100
# Number of bands that divide the signature matrix b. 
b = 100
# number of doc and col
doc = data.shape[0]
col = data.shape[1]
# define the sequence [1:1000000]
seq = np.arange(col)
#define the h
permutation_matrix = np.zeros((b,col))
for i in range(b):
    permutation_matrix[i,:] = np.random.permutation(seq)

print(permutation_matrix)


### Signature matrix M

In [ ]:
from tqdm import tqdm
# M 行为 hash函数的个数，列为文件数
M = np.ones((b,doc))
M = -1*M

# 对于第i个hash序列，第j篇文章进行操作
# 遍历col个k-shingles，第k个如果等于1，更新min_index
for i in tqdm(range(b)):
    hi = permutation_matrix[i,:]
    for j in range(doc):
#        min_index = 300
#         for k in range(col):
#             if data[j,k] == 1:
#                 if min_index > permutation_matrix[i,k]:
#                     min_index = permutation_matrix[i,k]
        data_j = data[j,:]
        min_index = min(hi[data_j==1])
        M[i,j] = min_index
print(M)


the next code cell is reference to 
> https://github.com/guoziqingbupt/Locality-sensitive-hashing

In [ ]:
import random
import hashlib

def minHash(sigMatrix, b, r):

    hashBuckets = {}
    
    # begin and end of band row
    begin, end = 0, r
        
    # count the number of band level
    count = 0
    doc = sigMatrix.shape[1]
    while end <= sigMatrix.shape[0]:
        print(end)
        # traverse the column of sig matrix
        for colNum in range(doc):

            # generate the hash object, we used md5
            hashObj = hashlib.md5()

            # calculate the hash value
            band = str(sigMatrix[begin: begin + r, colNum])
            hashObj.update(band.encode())

            # use hash value as bucket tag
            tag = hashObj.hexdigest()

            # update the dictionary
            if tag not in hashBuckets:
                hashBuckets[tag] = [colNum]
            elif colNum not in hashBuckets[tag]:
                hashBuckets[tag].append(colNum)
        begin += r
        end += r


    # return a dictionary
    return hashBuckets

In [ ]:
band = 20
row = 5
hashBucket = minHash(M,band,row)


In [ ]:
print(len(hashBucket))

Problem: For document 0 (the one with id '0'), list the **30** most similar document ids (except document 0 itself). You can valid your results with the [sklearn.metrics.jaccard_score()](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.jaccard_score.html) function.

Tips: You can adjust your parameters to hash the documents with similarity *s > 0.8* into the same bucket.

In [ ]:
# Your code here, you can add cells if necessary
from sklearn.metrics import jaccard_score

#cnt表示中第i个元素表示id = i的文件和id = 0的文件中被哈希到相同bucket的band个数
cnt = np.zeros(doc)
print(cnt.shape)
queryCol = 0
#检查每一个哈希值，如果bucket中的hash值对应的集合中含有“0”，那么将该集合中的其他文章的计数+1
for key in hashBucket:
    if queryCol in hashBucket[key]:
        for i in hashBucket[key]:
            cnt[i] += 1

#排序找到cnt最大30个数的索引位置
sorted_cnt = np.argsort(cnt)

#sim_doc中保存文件id，sim中保存相似度
sim_doc = []
sim = []

for i in range(30):
    index = sorted_cnt[doc-i-1]
    tmp = jaccard_score(data[index],data[0])
    sim_doc.append(index)
    sim.append(tmp)
    
print(sim_doc)
print(sim)